In [1]:
import numpy as np
import pandas as pd
import haversine
import math

In [57]:
fn = "../data/deped_school_location_with_enrolment_2017_2018_geocoded2_intersection.csv"
df = pd.read_csv(fn)

In [58]:
# read in raw dataset
fn2 = "../data/deped_school_location_with_enrolment_2017_2018.csv"
df2 = pd.read_csv(fn2)

In [59]:
fn_out = "../data/deped_school_location_with_enrolment_2017_2018_geocoded2_validationB.csv"

In [61]:
len(df), len(df2), len(df)*1./502

(483, 47358, 0.9621513944223108)

In [62]:
df.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'Reg_Code', u'Reg_Name', u'Pro_Code', u'Pro_Name',
       u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name', u'MATCH_BARANGAY',
       u'MATCH_MUNICIPALITY', u'MATCH_REGION', u'SCHOOL NAME2', u'ADDRESS1',
       u'ADDRESS2', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code_2', u'Reg_Name_2',
       u'Pro_Code_2', u'Pro_Name_2', u'Mun_Code_2', u'Mun_Name_2',
       u'Bgy_Code_2', u'Bgy_Name_2'],
      dtype='object')

In [63]:
df.head(2)

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,GMAP_LON2,GMAP_DELTA_KM,Reg_Code_2,Reg_Name_2,Pro_Code_2,Pro_Name_2,Mun_Code_2,Mun_Name_2,Bgy_Code_2,Bgy_Name_2
0,100850,NANGALISAN ES,"-, BRGY. NANGALISAN, BACNOTAN",REGION I,LA UNION,BACNOTAN,NANGALISAN,16.434103,120.479775,1,...,120.353640,0.106399,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013303000,BACNOTAN,PH013303025,Nangalisan
1,100864,LIBBO ES,"LIBBO, BAGULIN ,LA UNION, BRGY. LIBBO, BAGULIN",REGION I,LA UNION,BAGULIN,LIBBO,16.570025,120.524387,1,...,120.519274,2.805844,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013304000,BAGULIN,PH013304006,Libbo


In [64]:
# convert to upper case
for col in "REGION", "MUNICIPALITY", "BARANGGAY", "Bgy_Name_2", "Mun_Name_2", "Reg_Name_2":
    df[col] = df[col].str.upper()

In [65]:
# compare strings - check if one is a substring of the other
df["MATCH_BARANGAY"] = np.nan
df["MATCH_MUNICIPALITY"] = np.nan
df["MATCH_REGION"] = np.nan
col1 = ["Bgy_Name_2", "Mun_Name_2", "Reg_Name_2"]
col2 = ["BARANGGAY", "MUNICIPALITY", "REGION"]
col_out = ["MATCH_BARANGAY", "MATCH_MUNICIPALITY", "MATCH_REGION"]
for i in np.arange(len(df)):
    row = df.loc[i]
    for j in np.arange(3):
        str1, str2 = row[col1[j]], row[col2[j]]
        if (str1 in str2) or (str2 in str1):
            match = 1
        else:
            match = 0
        df.loc[i, col_out[j]] = match

In [66]:
df.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'Reg_Code', u'Reg_Name', u'Pro_Code', u'Pro_Name',
       u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name', u'MATCH_BARANGAY',
       u'MATCH_MUNICIPALITY', u'MATCH_REGION', u'SCHOOL NAME2', u'ADDRESS1',
       u'ADDRESS2', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code_2', u'Reg_Name_2',
       u'Pro_Code_2', u'Pro_Name_2', u'Mun_Code_2', u'Mun_Name_2',
       u'Bgy_Code_2', u'Bgy_Name_2'],
      dtype='object')

In [67]:
len(df), len(df)*1./502

(483, 0.9621513944223108)

In [68]:
df["MAPPED"].value_counts()

1    483
Name: MAPPED, dtype: int64

In [69]:
# define INVALID = 1, if both municipality and region does not match
df["INVALID"] = 0
df.loc[(df[col_out[2]]==0) & (df[col_out[1]]==0), "INVALID"] = 1

In [70]:
df["INVALID"].value_counts()

0    436
1     47
Name: INVALID, dtype: int64

In [79]:
47./483

0.09730848861283643

In [71]:
df[col_out[0]].value_counts(1)

0.0    0.534161
1.0    0.465839
Name: MATCH_BARANGAY, dtype: float64

In [72]:
df[col_out[1]].value_counts(1)

1.0    0.817805
0.0    0.182195
Name: MATCH_MUNICIPALITY, dtype: float64

In [73]:
df[col_out[2]].value_counts(1)

1.0    0.724638
0.0    0.275362
Name: MATCH_REGION, dtype: float64

In [74]:
df.head()

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,GMAP_LON2,GMAP_DELTA_KM,Reg_Code_2,Reg_Name_2,Pro_Code_2,Pro_Name_2,Mun_Code_2,Mun_Name_2,Bgy_Code_2,Bgy_Name_2
0,100850,NANGALISAN ES,"-, BRGY. NANGALISAN, BACNOTAN",REGION I,LA UNION,BACNOTAN,NANGALISAN,16.434103,120.479775,1,...,120.353640,0.106399,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013303000,BACNOTAN,PH013303025,NANGALISAN
1,100864,LIBBO ES,"LIBBO, BAGULIN ,LA UNION, BRGY. LIBBO, BAGULIN",REGION I,LA UNION,BAGULIN,LIBBO,16.570025,120.524387,1,...,120.519274,2.805844,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013304000,BAGULIN,PH013304006,LIBBO
2,101000,PALINA ES,"MARCOS HI-WAY, BRGY. PALINA, PUGO",REGION I,LA UNION,PUGO,PALINA,16.342535,120.493935,1,...,120.489945,7.624803,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013313000,ROSARIO,PH013313027,SAN JOSE
3,101008,CARUNOAN ES,"-, BRGY. CARUNUAN EAST, ROSARIO",REGION I,LA UNION,ROSARIO,CARUNUAN EAST,16.153600,120.271477,1,...,120.470286,3.863359,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013313000,ROSARIO,PH013313010,CARUNUAN EAST
4,101054,NAYBA ES,"-, BRGY. APAYAO, SAN GABRIEL",REGION I,LA UNION,SAN GABRIEL,APAYAO,16.642738,120.546997,1,...,120.548464,16.018998,PH010000000,REGION I (ILOCOS REGION),PH013300000,LA UNION,PH013315000,SAN GABRIEL,PH013315019,POBLACION


In [75]:
# merge with raw dataset 

In [76]:
df_out = pd.merge(df2, df[[u'SCHOOL ID', 'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1',
       u'GMAP_LON1', u'GMAP_LAT2', u'GMAP_LON2', u'GMAP_DELTA_KM',
       u'Reg_Code_2', u'Reg_Name_2', u'Pro_Code_2', u'Pro_Name_2', u'Mun_Code_2',
       u'Mun_Name_2', u'Bgy_Code_2', u'Bgy_Name_2', "MATCH_BARANGAY", "MATCH_MUNICIPALITY", "MATCH_REGION"]], on="SCHOOL ID", how="left")

In [77]:
df_out.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code_2', u'Reg_Name_2',
       u'Pro_Code_2', u'Pro_Name_2', u'Mun_Code_2', u'Mun_Name_2',
       u'Bgy_Code_2', u'Bgy_Name_2', u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY',
       u'MATCH_REGION'],
      dtype='object')

In [78]:
df_out.to_csv(fn_out, index=False)